In [152]:
!pip install scikit-surprise

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp38-cp38-win_amd64.whl size=1297767 sha256=694fb3cded548453337d931683dbe754dd8271cb67e3b855983ef8233bca9387
  Stored in directory: c:\users\ruiqi\appdata\local\pip\cache\wheels\99\34\23\b19f7de7352af5d1913f2654641432d6740c2666eecf2472ea
Successfully built scikit-surprise
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
  Attempting uninstall: joblib
    Found existin

In [34]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
#from sklearn import svm
import numpy
import string
import random
import string
#from sklearn import linear_model
import warnings
from surprise import SVDpp, Reader, Dataset
from surprise.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [62]:
!pip install implicit

In [84]:
!pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl (1.9 kB)


ERROR: Could not find a version that satisfies the requirement tensorflow-intel==2.13.1; platform_system == "Windows" (from tensorflow) (from versions: 0.0.1, 2.10.0.dev20220728, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0)
ERROR: No matching distribution found for tensorflow-intel==2.13.1; platform_system == "Windows" (from tensorflow)


In [63]:
import pandas as pd
from implicit import bpr

In [7]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [8]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [9]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [10]:
# Some data structures that will be useful

In [11]:
allRatings = []
userIDs = {}
bookIDs = {}
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)
    if not l[0] in userIDs: userIDs[l[0]] = len(userIDs)
    if not l[1] in bookIDs: bookIDs[l[1]] = len(bookIDs)

In [12]:
len(allRatings)

200000

In [13]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
usersPerBook = defaultdict(set) # Maps an item to the users who rated it
booksPerUser = defaultdict(set) # Maps a user to the items that they rated
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    usersPerBook[b].add(u)
    booksPerUser[u].add(b)

In [14]:
ratingsValid[0]

('u47877739', 'b50020691', 4)

In [15]:
##################################################
# Read prediction                                #
##################################################

In [16]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [17]:
### Question 1

In [18]:
unseenPerUser = defaultdict(list)
#uniqueUser = []
for u in userIDs: #for every user   
    for b in bookIDs: #find a book
            if u not in booksPerUser or b not in booksPerUser[u]: #the user hasn't seen
                unseenPerUser[u].append(b)

In [19]:
ratingsValidNeg = []
numpy.random.seed(0)
for u,b,r in ratingsValid:
    ratingsValidNeg.append((u,b,1)) #has read
    unseenList = unseenPerUser[u]
    bookNum = numpy.random.randint(len(unseenList))
    ratingsValidNeg.append((u, unseenList[bookNum], 0)) #has not read
random.shuffle(ratingsValidNeg)

In [20]:
predictions = []
ytest = [v[2] for v in ratingsValidNeg]
for l in ratingsValidNeg:
    if l[1] in return1:
        predictions.append(1)
    else:
        predictions.append(0)


In [21]:
### Question 2

In [108]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
threshold = totalRead/4*3
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > threshold: break

In [23]:
### Question 3/4

In [24]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [25]:
########################################## predict read #########################################

In [69]:
Xui = scipy.sparse.lil_matrix((len(userIDs), len(bookIDs)))
for d in allRatings:
    Xui[userIDs[d[0]],bookIDs[d[1]]] = 1
    
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [70]:
model = bpr.BayesianPersonalizedRanking(factors = 5)
model.fit(Xui_csr)

In [73]:
recommended = model.recommend(0, Xui_csr[0])

In [86]:
recommended

(array([1089,  431,    2, 1388,  490, 2633,  695,  328, 1105,  329]),
 array([0.74203694, 0.73789394, 0.7362339 , 0.731352  , 0.70485634,
        0.6932037 , 0.66025555, 0.65771985, 0.65729755, 0.65431243],
       dtype=float32))

In [101]:
# predictions = open("predictions_Read.csv", 'w')
# for l in open("pairs_Read.csv"):
#     if l.startswith("userID"):
#         predictions.write(l)
#         continue
#     u,b = l.strip().split(',')
#     maxSim=0
# #     if u in userIDs and b in bookIDs and len(ratingsPerUser[u])>1:
# #         ids, score = model.recommend(userIDs[u], Xui_csr[userIDs[u]], N=len(ratingsPerUser[u])/2)
# #         for i,s in zip(ids, score):
# #             if i==bookIDs[b]:
# #                 if s>0.5:
# #                     predictions.write(u + ',' + b + ",1\n")
# #                 else:
# #                     predictions.write(u + ',' + b + ",0\n")
# #                 continue
    
    
#     users = set(ratingsPerItem[b])
#     for b2,_ in ratingsPerUser[u]:
#         sim = Jaccard(users, set(ratingsPerItem[b2]))
#         if sim>maxSim:
#             maxSim = sim
#     if (maxSim > 0.013) or b in return1 or len(ratingsPerItem[b])>40:
#         predictions.write(u + ',' + b + ",1\n")
#     else:
#         predictions.write(u + ',' + b + ",0\n")

# predictions.close()

In [110]:
####################################### final implementation ###########################################

scorePerUser = defaultdict(list)
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    maxSim=0
#     if u in userIDs and b in bookIDs and len(ratingsPerUser[u])>1:
#         ids, score = model.recommend(userIDs[u], Xui_csr[userIDs[u]], N=len(ratingsPerUser[u])/2)
#         for i,s in zip(ids, score):
#             if i==bookIDs[b]:
#                 if s>0.5:
#                     predictions.write(u + ',' + b + ",1\n")
#                 else:
#                     predictions.write(u + ',' + b + ",0\n")
#                 continue
    
    
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Jaccard(users, set(ratingsPerItem[b2]))
        if sim>maxSim:
            maxSim = sim
    scorePerUser[u].append((maxSim, b))
    
for u in scorePerUser:
    scorePerUser[u].sort(key=lambda tup: tup[0], reverse=True)  
    count = 0
    for scores,b in scorePerUser[u]:
        if (count < int(len(scorePerUser[u])/2)) and (scores>0.015) or b in return1 or len(ratingsPerItem[b])>50:
            predictions.write(u + ',' + b + ",1\n")
        else: 
            predictions.write(u + ',' + b + ",0\n")
        count += 1
        
# for l in open("pairs_Read.csv"):
#     if l.startswith("userID"):
#         predictions.write(l)
#         continue
#     u,b = l.strip().split(',')
#     maxSim=0
#     median = scorePerUser[u][int(len(scorePerUser[u])/2)]
#     users = set(ratingsPerItem[b])
#     for b2,_ in ratingsPerUser[u]:
#         sim = Jaccard(users, set(ratingsPerItem[b2]))
#         if sim>maxSim:
#             maxSim = sim
    
#     if (maxSim >= median):
#         predictions.write(u + ',' + b + ",1\n")
#     else:
#         predictions.write(u + ',' + b + ",0\n")

predictions.close()

In [156]:
##################################################
# Rating prediction                              #
##################################################

In [157]:
### Question 6

In [115]:
#use ratingsValid and ratingsTrain
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
data = Dataset.load_from_file("train_Interactions.csv", reader=reader)

In [116]:
trainset, testset = train_test_split(data, test_size=1/20)

In [117]:
### Question 7

In [39]:
minBeta = model.bu[0]
maxBeta = model.bu[0]
userIndex = 0
minUser = trainset.to_raw_uid(0)
maxUser = trainset.to_raw_uid(0)

for u in trainset.all_users():
    b = model.bu[userIndex]
    if b<minBeta:
        minBeta = b
        minUser = trainset.to_raw_uid(u)
    if b>maxBeta:
        maxBeta = b
        maxUser = trainset.to_raw_uid(u)
    userIndex += 1

AttributeError: 'SVDpp' object has no attribute 'bu'

In [40]:
### Question 8

In [147]:
mses = []
lambdas = [0.008, 0.01, 0.013]
for lr in lambdas:
    model = SVDpp(reg_all=0.08, n_factors=3, lr_all=0.0075, lr_bu=0.007, lr_pu=lr, init_std_dev=0.053)
    model.fit(trainset)
    predictions = model.test(testset)
    sse = 0
    for p in predictions:
        sse += (p.r_ui - p.est)**2
    mses.append(sse / len(predictions))

validMSE = mses[0]
lamb = lambdas[0]
for (m,l) in zip(mses, lambdas):
    if m<validMSE:
        validMSE = m
        lamb = l

In [148]:
(lamb, validMSE)

(0.008, 1.4697120786577704)

In [179]:
answers['Q8']

(0.05, 1.4469116716494959)

In [205]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [111]:
model = SVDpp(reg_all=0.11, n_factors=3, lr_all=0.0075, n_epochs=100)
model.fit(trainset)
Rui = trainset.global_mean

In [112]:
########################################## predict rating #########################################

In [113]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    pred = model.predict(u, b, r_ui=Rui)
    predictions.write(u + ',' + b + ',' + str(pred.est) + '\n')
predictions.close()

In [154]:
f = gzip.open("steam_reviews.json.gz")
#dataset = []
for i, l in enumerate(f):
    if i==0:
        print(l)
        break

b"{u'username': u'Chaos Syren', u'hours': 0.1, u'products': 41, u'product_id': u'725280', u'page_order': 0, u'date': u'2017-12-17', u'text': u'This would not be acceptable as an entertainment even back in the day when these graphics were all there was to be had. No effort has been made to bring the player into any story or even entertain.', u'early_access': False, u'page': 1}\n"
